# Quantitative Trait Loci (QTL) analysis

## Introduction

This workshop focuses on large SNP data sets such as those obtained from genotyping-by-sequencing (GBS) for population genetic analysis in R. GBS is one of several techniques used to genotype populations using high throughput sequencing (HTS). In GBS, the genome is reduced in representation by using restriction enzymes, and then sequencing these products using HTS.

We will use a data set of 536 samples of the Lumpfish  (_Cyclopterus lumpus_). They are found in the cold waters of the Arctic, North Atlantic, and North Pacific oceans. There are used as cleaner fish by the Salmon farming industry.

Breeding families were obtained from 4 familles to investigate sex determination and gowth/length markers. A total of 536 samples were sequenced using the Illumina HiSeq 2500 technology with 150 bp paired-end reads and a target insert size of 600 bp. Currently, there is little information about the population structure of _C. lumpus_. The VCF data for this population can be downloaded from: [families.snps.vcf.gz](https://github.com/pseudogene/aqupgp3-workshops/raw/master/families.snps.vcf.gz).

To obtain variant calls in form of VCF data, the FASTQ reads from HTS were mapped to the reference genome of _C. lumpus_ using bowtie2 (Langmead & Salzberg, 2012). Variants were called using the GATK HaplotypeCaller (McKenna et al., 2010). Data was filtered as follows:

 * A minimum read-depth of 5x.
 * Only variants with a QC greater than 20 were retained.
 * Variants with more than 30% missing data were removed.

In addition to the VCF data, we have included the file `families.pop.tsv`, a tab-delimited text file that includes the name of the sample, country of origin, and the population from where it was sampled. The file is available for download at: [families.pop.tsv](https://github.com/pseudogene/aqupgp3-workshops/raw/master/families.pop.tsv). This link will likely open the data in a browser. Save the data onto your computer as an ASCII text file with the same name.

## Opening and examining the data

Let's first load the libraries needed for analysis:

In [ ]:
library(vcfR)
library(ape)
library(RColorBrewer)
library(tidyr)
library(ggplot2)

library(SNPassoc)

options(repr.plot.width=16, repr.plot.height=8)

Make sure you are in the right folder with the downloaded files available. Some of these packages will print a message when they are loaded. Here we suppressed this information. When you load these packages, you may see more output than presented here.

Next, let’s open the VCF file using vcfR and check that we have 536 samples and 10,630 SNPs:

In [ ]:
fish_data <- read.vcfR("families.snps.vcf.gz")

Once we’ve loaded the data into R, we can validate it by entering our object’s name in the console:

In [ ]:
fish_data

VCF data does not typically include any sort of population information. We have to load this data separately from the text-delimited file we downloaded above that includes the ID of samples and the state where the samples were obtained from. Let’s load the `fish.pop.tsv` file into memory by using the `read.table()` function:

In [ ]:
fish_pop <- read.delim("families.pop.tsv", header=TRUE,  sep="\t")

We can now check that all the samples in the population table.

In [ ]:
head(fish_pop)

In [ ]:
table(fish_pop$sex)
barplot(table(fish_pop$sex), las=3, ylab="Sample size");

Out of 536 individual only 287 (149 females and 138 males) were sexed.

## Converting the dataset to a SNPassoc object

The next step is to convert the data set into an object that is usable by `SNPassoc`. The `vcfR` package contains multiple functions to convert data into other formats (see the `converting_data` vignette of _vcfR_): `vignette("converting_data")`. For our particular purpose we want to convert the _vcfR_ object into a `SNPassoc` object. We can use the `vcfR2loci` function for this:

In [ ]:
fish_snp <- vcfR2loci(fish_data, return.alleles = TRUE)
fish_snp <- fish_snp[fish_pop$sample,]; # Keep only the individual sexed
fish_snp$sample <- rownames(fish_snp)
fish_snp <- as.data.frame(merge(fish_pop, fish_snp, by="sample"))

In [ ]:
fish_snp

Create an object for SNPassic: list of markers

In [ ]:
SNPAssoc <- setupSNP(data=fish_snp, colSNPs=7:length(fish_snp), sep='/')
colnames(fish_snp) = colnames(SNPAssoc)

We now end up with a SNPassic object of filtered VCF data:

In [ ]:
head(SNPAssoc)

## Descriptive analysis (preview)

We can verify that the SNP variables are given the new class snp

In [ ]:
summary(SNPAssoc$X1796459.76..)

which shows the genotype and allele frequencies for a given SNP, testing for Hardy-Weinberg equilibrium (HWE). We can also visualize the results in a plot by

In [ ]:
plot(SNPAssoc$X1796459.76..)

The argument type helps to get a pie chart

In [ ]:
plot(SNPAssoc$X1796459.76.., type=pie)

## Hardy-Weinberg equilibrium

Genotyping of SNPs needs to pass quality control measures. Aside from technical details that need to be considered for filtering SNPs with low quality, genotype calling error can be detected by a HWE test. The test compares the observed genotype frequencies with those expected under random mating, which follows when the SNPs are in the absence of selection, mutation, genetic drift, or other forces. Therefore, HWE must be checked only in controls. There are several tests described in the literature to verify HWE. In SNPassoc HWE is tested for all the bi-allelic SNP markers using a fast exact test Wigginton, Cutler, and Abecasis (2005) implemented in the tableHWE function.

In [ ]:
hwe <- tableHWE(SNPAssoc)
head(hwe)

We observe that the first SNPs in the dataset are under HWE since their P-values rejecting the HWE hypothesis (null hypothesis) are larger than 0.05.

Notice that one is interested in keeping those SNPsthat do not reject the null hypothesis. As several SNPs are tested, multiple comparisons must be considered. In this particular setting, a threshold of 0.001 is normally considered. As a quality control measure, it is not necessary to be as conservative as in those situations where false discovery rates need to be controlled.

SNPs that do not pass the HWE test must be removed form further analyses. We can recall `setupSNP` and indicate the columns of the SNPs to be kept

In [ ]:
snps.ok <- rownames(hwe)[hwe[,1]>=0.001]
pos <- which(colnames(fish_snp) %in% snps.ok, useNames = FALSE)
length(pos)

In [ ]:
head(pos)

In [ ]:
#SNPAssoc <- setupSNP(data=fish_snp, colSNPs=pos, sep='/')

Note that in the variable pos we redefine the SNP variables to be considered as class `snp`.

## SNP association analysis

We are interested in finding those SNPs associated with sex differentiation that is encoded in the variable `sex`. We first illustrate the association between phenotypic sex and the SNP `X1796459.76..`. The association analysis for all genetic models is performed by the function association that regresses `sex` on the variable `X1796459.76..` from the dataset `SNPAssoc` that already contains the SNP variables of class snp.

In [ ]:
association(sex ~ X1796459.76.., data = SNPAssoc)

For multiple SNP data, our objective is to identify the variants that are significantly associated with the trait. The most basic strategy is, therefore, to fit an association test like the one described above for each of the SNPs in the dataset and determine which of those associations are significant. The massive univariate testing is the most widely used analysis method for omic data because of its simplicity. In SNPassoc, this type of analysis is done with the function WGassociation

In [ ]:
SNPAssocSex <- WGassociation(sex, data=SNPAssoc, model='codominant')
head(SNPAssocSex)

The P-values obtained from massive univariate analyses are visualized with the generic plot function

In [ ]:
summary(SNPAssocSex)

In [ ]:
Bonferroni.sig(SNPAssocSex, model = 'codominant', alpha = 0.05)

In [ ]:
## plot does not work in R/jupyter
#plot(SNPAssocSex)

toplot <- data.frame(snp=rownames(SNPAssocSex), P=SNPAssocSex$codominant)
ggplot(toplot, aes(x = snp, y = -log10(P), size = -log10(P))) + geom_point(alpha = 0.75) + geom_hline(yintercept = -log10(5e-8), color = "grey40", linetype = "dashed")

Look at the best marker details

In [ ]:
association(sex ~ X1804870.103.., data = SNPAssoc)

Recover the genotypes of interest

In [ ]:
SNPAssoc[,c("sample","sex","X1804870.103..")]

In [ ]:
ggplot(SNPAssoc[SNPAssoc$X1804870.103..!='-',], aes(X1804870.103.., sex)) + geom_jitter(width = 0.1, height=0.1, aes(colour=factor(sex)), alpha=0.6, size=2) + theme_bw()

## Further analysis

What about the `Total_length`, `Standard_length` and `Weight`
...

In [ ]:
SNPAssocW <- WGassociation(Weight, data=SNPAssoc, model='codominant')
summary(SNPAssocW)
toplot <- data.frame(snp=rownames(SNPAssocW), P=SNPAssocW$codominant)
ggplot(toplot, aes(x = snp, y = -log10(P), size = -log10(P))) + geom_point(alpha = 0.75) + geom_hline(yintercept = -log10(5e-8), color = "grey40", linetype = "dashed")